In [1]:
import sys
import os.path as osp
import pprint
import networkx as nx
from tqdm import tqdm
from networkx.algorithms import bipartite
import numpy as np
import pandas as pd
import torch

SRC_SUBDIR = '/app/src/'
SRC_SUBDIR = osp.abspath(SRC_SUBDIR)
if SRC_SUBDIR not in sys.path:
    print(f'Adding source directory to the sys.path: {SRC_SUBDIR!r}')
    sys.path.insert(1, SRC_SUBDIR)
pprint.pprint(sys.path)

from util import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Adding source directory to the sys.path: '/app/src'
['/app/notebooks',
 '/app/src',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages']


In [2]:
df = load_binary('/app/data/df_with_term.pkl')
company_node = load_binary('/app/data/company_node.pkl')
term_node = load_binary('/app/data/term_node.pkl')

In [3]:
def make_patent_any_relation(df, column_name, node_list):
    table = str.maketrans({
    '\u3000': '',
    ' ': '',
    '\t': ''
    })

    data = torch.tensor(np.zeros((len(df), len(node_list))), dtype=torch.float32)

    for i in tqdm(range(len(df))):
        targets = df[column_name].values[i].split('|')
        for t in targets:
            t = t.translate(table)
            if t in node_list:
                data[i, node_list.index(t)] = 1
    
    return data

In [4]:
patent_company = make_patent_any_relation(df, 'company', company_node)
save_binary(patent_company, '/app/data/patent_company.pkl')

100%|██████████| 44564/44564 [00:00<00:00, 48255.20it/s]


'Done'

In [5]:
patent_term = make_patent_any_relation(df, 'term_all', term_node)
save_binary(patent_term, '/app/data/patent_term.pkl')

100%|██████████| 44564/44564 [00:38<00:00, 1164.12it/s]


'Done'

In [6]:
cpt = torch.matmul(patent_company.T, patent_term)
cpt = clamp(cpt, 0, 1)
cpt = cpt.to('cpu')
save_binary(cpt, '/app/data/cpt.pkl')

'Done'

In [7]:
bi_adj = load_binary('/app/data/bipartite_adj.pkl')

In [8]:
H = load_binary('/app/data/bipartite_graph.pkl')

In [19]:
term_centrality = nx.eigenvector_centrality(H, max_iter=1000000)
term_centrality = sorted(term_centrality.items(), key=lambda x:x[1], reverse=True)
for k, v in term_centrality[:30]:
    print(k, v)
print('------------')


清水建設株式会社 0.1539085628409875
大成建設株式会社 0.15356611724828387
鹿島建設株式会社 0.15338234995035466
株式会社大林組 0.15026749858230806
株式会社竹中工務店 0.13913057196744666
前田建設工業株式会社 0.12199091899304211
株式会社奥村組 0.11914946533494357
株式会社熊谷組 0.11771024565783735
西松建設株式会社 0.1154283814160206
戸田建設株式会社 0.11460623095756382
五洋建設株式会社 0.11336862638209189
三井住友建設株式会社 0.11267242403908014
株式会社フジタ 0.1109405784036741
株式会社安藤・間 0.11043151401663431
東急建設株式会社 0.09541070609854106
佐藤工業株式会社 0.08365465375225116
飛島建設株式会社 0.08348063429463459
東亜建設工業株式会社 0.08128538281147228
オリエンタル白石株式会社 0.07822453796037747
鉄建建設株式会社 0.07812842093508236
株式会社竹中土木 0.07514179283793129
東洋建設株式会社 0.07479791326720432
株式会社ピーエス三菱 0.07142765333381737
株式会社不動テトラ 0.07110711270226121
高砂熱学工業株式会社 0.06756692413898507
技術分野 0.06478964168919338
株式会社長谷工コーポレーション 0.0646303470678972
岡部株式会社 0.0643913615179751
東日本旅客鉄道株式会社 0.06335524675843819
日本製鉄株式会社 0.06316891120401864
------------


In [10]:
cpt

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cpu')